In [ ]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

from tqdm import tqdm
import pytrec_eval
from statistics import mean
import json
import sys

In [4]:
from collections import defaultdict
qrel_file = "../msmarco/qrels.train.tsv"

qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)

fo = open("../msmarco/queries.train.labeled.tsv", "w")

with open('../msmarco/queries.train.tsv') as f:
    for line in f:
        if line.split("\t")[0] in qrels:
            fo.write(line)
            fo.flush()
            
fo.close()

In [ ]:
with open("output_64x1_0.01_0.088_warmup6000_100000_dev.run.json") as fo:  #"output_10000_dev.run.json", "w") as fo:
    line = fo.readline()
    

In [ ]:
import json
run = json.loads(line)

In [ ]:
import pytrec_eval
from statistics import mean
VALIDATION_METRIC = 'recip_rank'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run)
print(mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))


In [ ]:
from collections import defaultdict
bmtops = defaultdict(dict)
with open("../msmarco/wentai_splade_dev_top1000.tsv") as f:
    for line in f:
        qid, did, qtext, dtext = line[:-1].split("\t")
        bmtops[qid][did] = 1


with open("../msmarco/top1000.dev") as f, open("../msmarco/bm25-splade_dev_top1000.tsv", 'w') as fo:
    for line in f:
        qid, did, qtext, dtext = line[:-1].split("\t")
        if did not in bmtops[qid]:
            fo.write(line)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(num_evals)

In [ ]:
import numpy as np
np.min(num_evals)

In [ ]:
queries = dict()

with open("../msmarco/msmarco-passagetest2020-top1000_spladedoc3_10.tsv") as f:
    for line in f:
        qid, _, qtext, _ = line.strip().split("\t")
        queries[qid] = qtext
        
with open("../msmarco/wentai_splade_2020_top1000.tsv") as f, open("../msmarco/wentai_splade_2020_top1000_spladedoc3_10.tsv", "w") as fo:
    for line in f:
        qid, did, qtext, dtext = line.strip().split("\t")
        fo.write(f"{qid}\t{did}\t{queries[qid]}\t{dtext}\n")

In [ ]:
from collections import defaultdict
import pytrec_eval
from statistics import mean

VALIDATION_METRIC = 'recip_rank'    #'recip_rank' #'ndcg_cut_10' 
top_k = 10

qrel_file = "../msmarco/qrels.dev.tsv" #"../msmarco/qrels.dev.tsv"  f"../msmarco/20{yr}qrels-pass.txt"
qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)

run2 = defaultdict(dict)
with open("output_dev-on-splade_colspla_from_colbertspla20_3e-6_negpersys5_200k.tsv") as f: #f"output_{yr}_colspla_from_colbert_3e-6_negpersys20_200k.run.json.tsv"
    for line in f:
        qid, did, score = line.strip().split("\t")
        run2[qid][did] = float(score)

top_run = defaultdict(dict)
for q in run2:
    docs = sorted(run2[q].items(), key=lambda x: -x[1])
    for item in docs[:top_k]:
        top_run[q][item[0]] = item[1]
                    
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores2 = trec_eval.evaluate(top_run)
print(VALIDATION_METRIC, top_k, mean([d[VALIDATION_METRIC] for d in eval_scores2.values()]))

In [ ]:
from collections import defaultdict
import pytrec_eval
from statistics import mean

VALIDATION_METRIC = 'ndcg_cut_10'     #'recip_rank' #'ndcg_cut_10' 
top_k = 10

#for yr in ['19','20']:
yr = 19
qrel_file = f"../msmarco/20{yr}qrels-pass.txt" #"../msmarco/qrels.dev.tsv"  f"../msmarco/20{yr}qrels-pass.txt"
qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)

run = defaultdict(list)
drun = defaultdict(dict)
with open(f"../msmarco/wentai_splade_20{yr}.trec") as f: #f"../msmarco/wentai_splade_20{yr}.trec", "../msmarco/wentai_splade_dev_top1000.trec"
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        run[qid].append([did, -int(rank)])
        drun[qid][did] = -float(rank)

dsub_run = defaultdict(dict)
sub_run = defaultdict(list)
with open(f"results/output_{yr}-on-splade_colbert_271k.run.json.tsv") as f: #f"output_{yr}_colspla_from_colbert_3e-6_negpersys20_200k.run.json.tsv"
    for line in f:
        qid, did, score = line.split("\t")
        sub_run[qid].append([did, float(score)])
        dsub_run[qid][did] = float(score)

drun2 = defaultdict(dict)
run2 = defaultdict(list)
with open(f"results/output_{yr}-on-splade_colspla_from_colbert_3e-6_negpersys5_200k.run.json.tsv") as f: #f"output_{yr}_colspla_from_colbert_3e-6_negpersys20_200k.run.json.tsv"
    for line in f:
        qid, did, score = line.split("\t")
        run2[qid].append([did, float(score)])
        drun2[qid][did] = float(score)

for q in sub_run:
    sub_run[q] = sorted(sub_run[q], key=lambda x: -x[1])

for q in run:
    run[q] = sorted(run[q], key=lambda x: -x[1])

for q in run2:
    run2[q] = sorted(run2[q], key=lambda x: -x[1])


    
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores1 = trec_eval.evaluate(drun)
print(yr, "splade", mean([d[VALIDATION_METRIC] for d in eval_scores1.values()]))

eval_scores2 = trec_eval.evaluate(dsub_run)
print(yr, "colbert", mean([d[VALIDATION_METRIC] for d in eval_scores2.values()]))

eval_scores3 = trec_eval.evaluate(drun2)
print(yr, "colspla", mean([d[VALIDATION_METRIC] for d in eval_scores3.values()]))


In [ ]:
queries = dict()
docs = dict()

with open("../msmarco/wentai_splade_2019_top1000.tsv") as f:
    for line in f:
        qid, did, qtext, dtext = line.strip().split("\t")
        queries[qid] = qtext
        docs[did] = dtext

In [ ]:
eval_all = dict()
for k in eval_scores1:
    eval_all[k] = {"splade": eval_scores1[k]['ndcg_cut_10'],
                  "colbert": eval_scores2[k]['ndcg_cut_10'],
                  "colspla": eval_scores3[k]['ndcg_cut_10']}

In [ ]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from collections import defaultdict
from tqdm import tqdm
import os
import pytrec_eval
from collections import OrderedDict, defaultdict
from statistics import mean
import json
import sys
from training_with_sentence_transformers.models import MLMTransformerDense, ColBERTTransformer
from training_with_sentence_transformers.losses import pairwise_dot_score
import nvidia_smi

def _split_into_batches(features, bsize):
    batches = []
    for offset in range(0, features["input_ids"].size(0), bsize):
        batches.append({key: features[key][offset:offset+bsize] for key in features.keys()})

    return batches

nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

agg = "max"
bsize = 16
model_type_or_dir = "training_with_sentence_transformers/output/colbert_hn_[q][d]_num20_marginkldiv_position5-batch_size_16-2022-07-06_18-01-08/140000/0_ColBERTTransformer" #"colspla-prf_from_colbert_3e-6_negpersys5" #"output/0_MLMTransformer"


topk = open("../msmarco/wentai_splade_2019_top1000.tsv")

VALIDATION_METRIC = 'ndcg_cut_10' 

qrel_file = "../msmarco/2019qrels-pass.txt"

qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)



dense_weight = 0.0
model = ColBERTTransformer(model_type_or_dir, max_seq_length=256)
checkpoint = torch.load(os.path.join(model_type_or_dir, "checkpoint.pt"), map_location='cpu')
model.load_state_dict(checkpoint)
    
model.eval()
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
tokens = ["[unused0]", "[unused1]", "[unused2]"] #[unused0] for query, [unused1] for doc, [unused2] for query expansion
tokenizer.add_tokens(tokens, special_tokens=True)
# top k inference               

In [ ]:
run = defaultdict(dict)
run_prf = defaultdict(dict)
topk_prf = open("../msmarco/wentai_splade_2019_top1000_spladedoc3_10.tsv")

cur_qid = None
cur_qtext = None
cur_qtext_prf = None
dtexts = []
dids = []

for idx, line in enumerate(tqdm(topk_prf)):
    qid, didt, qtext, dtext = line.strip().split("\t")
    qtext_prf = qtext.split("[unused2] ")[1]
    qtext = qtext.split("[SEP]")[0]
    if len(qrels[qid]) == 0:
        continue
    if cur_qid is not None and qid != cur_qid:
        with torch.no_grad():
            q_features_prf = tokenizer("[unused0] " + cur_qtext_prf, return_tensors="pt").to('cuda')
            q_features_prf = model(q_features_prf)
            token_rep_q_prf = torch.nn.functional.normalize(q_features_prf['last_layer_embeddings'], p=2, dim=2)
            
            q_features = tokenizer("[unused0] " + cur_qtext, return_tensors="pt").to('cuda')
            q_features = model(q_features)
            token_rep_q = torch.nn.functional.normalize(q_features['last_layer_embeddings'], p=2, dim=2)
            
            del q_features_prf, q_features
            
            d_features = tokenizer(["[unused1] " + dtext for dtext in dtexts], return_tensors="pt", max_length=256,truncation=True,padding=True)
            d_features = _split_into_batches(d_features,bsize=bsize)
            
            all_scores_prf = []
            all_scores = []
            for batch in d_features:
                d_batch = model(batch)
                d_mask = d_batch['attention_mask'].to('cuda')
                d_emb = d_batch['last_layer_embeddings']
                del d_batch
                d_mask = d_mask.unsqueeze(-1)
                token_rep_d =  d_emb * d_mask
                del d_mask, d_emb
                token_rep_d = torch.nn.functional.normalize(token_rep_d)
                token_rep_d = token_rep_d.permute(0,2,1)
                inter =  (token_rep_q @ token_rep_d)
                scores = inter.max(2).values.sum(1).tolist()
                del inter
                inter = token_rep_q_prf @ token_rep_d
                scores_prf =  inter.max(2).values.sum(1).tolist()
                del inter
                del token_rep_d
                torch.cuda.empty_cache()
                all_scores_prf.extend(scores_prf)
                all_scores.extend(scores)
                
            for did, score in zip(dids, all_scores):
                run[cur_qid][did] = score
                
            for did, score in zip(dids, all_scores_prf):
                run_prf[cur_qid][did] = score
            
            del token_rep_q_prf, token_rep_q
            torch.cuda.empty_cache()
            dtexts = []
            dids = []
        
    cur_qid = qid
    cur_qtext = qtext
    dtexts.append(dtext)
    dids.append(didt)
    cur_qtext_prf = qtext_prf

In [ ]:
def combine_run(run1, run2, weight2 = 0.1):
    run = defaultdict(dict)
    for q in run1:
        for d in run1[q]:
            run[q][d] = run1[q][d] + weight2 * run2[q][d]
    return run

In [ ]:
run10 = combine_run(run, run_prf, 0.1)
run5 = combine_run(run, run_prf, 0.05)
run20 = combine_run(run, run_prf, 0.2)
run30 = combine_run(run, run_prf, 0.3)

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run_prf)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run10)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run5)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run20)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

In [ ]:
VALIDATION_METRIC = 'ndcg_cut_10'
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(run30)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])

# 如果把prf当单独的，和其他两个合起来

In [ ]:
from collections import defaultdict
import pytrec_eval
from statistics import mean
import scipy.stats as stats
yr = '19'
k1=60
k2=60
k3=200

frun = defaultdict(dict)

with open(f"../msmarco/wentai_splade_20{yr}.trec") as f: #f"../msmarco/wentai_splade_20{yr}.trec", "../msmarco/wentai_splade_dev_top1000.trec"
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        frun[qid][did] = 1.0/(int(rank) + k1)
run_retriever = {q: list(frun[q].items()) for q in frun}

run_ranker = {q: list(run[q].items()) for q in run}
for q in run_ranker:
    run_ranker[q] = sorted(run_ranker[q], key=lambda x: -x[1])

for q in run_ranker:
    for idx, item in enumerate(run_ranker[q]):
        if item[0] in run_ranker[q]:
            frun[q][item[0]] += 1.0/(int(idx+1) + k2)
        else:
            frun[q][item[0]] = 1.0/(int(idx+1) + k2)

trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(frun)
print("hybrid", mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))


    
run_ranker_prf = {q: list(run_prf[q].items()) for q in run_prf}
for q in run_ranker_prf:
    run_ranker_prf[q] = sorted(run_ranker_prf[q], key=lambda x: -x[1])

for q in run_ranker_prf:
    
    x1 = [x[0] for x in run_retriever[q]]
    x2 = [x[0] for x in run_ranker[q]]
    tau, p_value = stats.kendalltau(x1[:10], x2[:10])

    if tau < -0.05:
        k3 = 60
        print(q)
    else:
        continue
        
    for idx, item in enumerate(run_ranker_prf[q]):
        if item[0] in frun[q]:
            frun[q][item[0]] += 1.0/(int(idx+1) + k3)

trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(frun)
mean([d[VALIDATION_METRIC] for d in eval_scores.values()])


In [ ]:
import scipy.stats as stats
for q in run_retriever:
    x1 = [x[0] for x in run_retriever[q]]
    x2 = [x[0] for x in run_ranker[q]]
    tau, p_value = stats.kendalltau(x1[:10], x2[:10])
    print(q, tau, p_value)

In [ ]:
with open("../msmarco/wentai_splade_dev_top1000.tsv") as f, open("../msmarco/wentai_splade_dev_top1000_spladedoc3_10.tsv", "w") as fo:
    for line in f:
        qid, did, _, dtext = line.split("\t")
        fo.write("\t".join([qid, did, queries[qid], dtext]))

In [ ]:
queries = dict()
with open('../msmarco/queries.dev.spladedoc3_10.tsv') as f:
    for line in f:
        qid, qtext = line[:-1].split("\t")
        queries[qid] = qtext

In [ ]:
import gzip
import json
top_docs = dict()
with gzip.open("../msmarco/dev_top3_docs_original.jsonl.gz") as f:
    for line in f:
        info = json.loads(line)
        top_docs[info['id']] = info

In [ ]:
from collections import defaultdict
run = defaultdict(list)
with open("../msmarco/wentai_splade_dev_top1000.trec") as f:
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        if int(rank) <= 3: 
            run[qid].append(did)

In [ ]:
def combine_dicts(dict1, dict2, dict3):
    results = dict()
    for k in dict1:
        results[k] = dict1[k]
          
    for k in dict2:
        if k in results:
            results[k] = max(results[k], dict2[k])
        else:
            results[k] = dict2[k]
            
    for k in dict3:
        if k in results:
            results[k] = max(results[k], dict3[k])
        else:
            results[k] = dict3[k]
            
    return sorted([(k,v) for k,v in results.items()], key = lambda x: -x[1])[:10]

In [ ]:
with open('../msmarco/queries.dev.spladedoc3_10.tsv', 'w') as f:
    for q in run:
        r = combine_dicts(top_docs[int(run[q][0])]['vector'], 
                      top_docs[int(run[q][1])]['vector'],
                      top_docs[int(run[q][2])]['vector'])
        f.write(f"{q}\t{queries[q]} [SEP] [unused2] {' '.join([x[0] for x in r])}\n")
        f.flush()

In [ ]:
#FROM Sentence-BERT(https://github.com/UKPLab/sentence-transformers/blob/afee883a17ab039120783fd0cffe09ea979233cf/examples/training/ms_marco/train_bi-encoder_margin-mse.py) with minimal changes.
#Original License Apache2, NOTE: Trained MSMARCO models are NonCommercial (from dataset License)

import sys
import json
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, evaluation, InputExample
from sbert import SentenceTransformerA
import models
import logging
from datetime import datetime
import gzip
import os
import tarfile
import tqdm
from torch.utils.data import Dataset
import random
from shutil import copyfile
import pickle
import argparse
import losses
import torch
from collections import defaultdict
from data import MSMARCODataset
#from colbert_model import DETeacher

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])



train_batch_size = 16  # Increasing the train batch size generally improves the model performance, but requires more GPU memory
model_name = "training_with_sentence_transformers/output/colbert_hn_[q][d]_num20_marginkldiv_position5-batch_size_16-2022-07-06_18-01-08/140000/0_ColBERTTransformer"
max_passages = 512
ce_score_margin = 0
max_seq_length = 256  # Max length for passages. Increasing it implies more GPU memory needed
num_negs_per_system = 20 # We used different systems to mine hard negatives. Number of hard negatives to add from each system
num_epochs = 10  # Number of epochs we want to train

# Load our embedding model

word_embedding_model = models.ColBERTTransformer(model_name, max_seq_length=max_seq_length)
print(len(word_embedding_model.tokenizer))
tokens = ["[unused0]", "[unused1]", "[unused2]"] #[unused0] for query, [unused1] for doc, [unused2] for query expansion
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
print(len(word_embedding_model.tokenizer))

checkpoint = torch.load(os.path.join(model_name, "checkpoint.pt"), map_location='cpu')
word_embedding_model.load_state_dict(checkpoint)

model = SentenceTransformerA(modules=[word_embedding_model])
model_save_path = f'output/test' #_prfspladedoc310

# Write self to path
os.makedirs(model_save_path, exist_ok=True)

train_script_path = os.path.join(model_save_path, 'train_script.py')
copyfile(__file__, train_script_path)
with open(train_script_path, 'a') as fOut:
    fOut.write("\n\n# Script was called via:\n#python " + " ".join(sys.argv))

### Now we read the MS MARCO dataset

data_folder = '../../msmarco'
#data_folder = '/home/ec2-user/ebs/msmarco'


#### Read the corpus file containing all the passages. Store them in the corpus dict
corpus = {}  # dict in the format: passage_id -> passage. Stores all existing passages
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    tar_filepath = os.path.join(data_folder, 'collection.tar.gz')
    if not os.path.exists(tar_filepath):
        logging.info("Download collection.tar.gz")
        util.http_get('https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz', tar_filepath)

    with tarfile.open(tar_filepath, "r:gz") as tar:
        tar.extractall(path=data_folder)

logging.info("Read corpus: collection.tsv")
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        pid = int(pid)
        corpus[pid] = passage

### Read the train queries, store in queries dict
queries = {}  # dict in the format: query_id -> query. Stores all training queries
queries_filepath = os.path.join(data_folder, 'queries.train.tsv') #'queries.train.tsv'  'queries.train.qspladev2.top10.tsv'  'queries.train.spladedoc3_10.tsv'
if not os.path.exists(queries_filepath):
    tar_filepath = os.path.join(data_folder, 'queries.tar.gz')
    if not os.path.exists(tar_filepath):
        logging.info("Download queries.tar.gz")
        util.http_get('https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz', tar_filepath)

    with tarfile.open(tar_filepath, "r:gz") as tar:
        tar.extractall(path=data_folder)

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        qid = int(qid)
        queries[qid] = query


ce_scores_file = os.path.join(data_folder, 'cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz')
if not os.path.exists(ce_scores_file):
    logging.info("Download cross-encoder scores file")
    util.http_get('https://huggingface.co/datasets/sentence-transformers/msmarco-hard-negatives/resolve/main/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz', ce_scores_file)

# As training data we use hard-negatives that have been mined using various systems
hard_negatives_filepath = os.path.join(data_folder, 'msmarco-hard-negatives-splade.jsonl.gz')
if not os.path.exists(hard_negatives_filepath):
    logging.info("Download cross-encoder scores file")
    util.http_get('https://huggingface.co/datasets/sentence-transformers/msmarco-hard-negatives/resolve/main/msmarco-hard-negatives.jsonl.gz', hard_negatives_filepath)


train_queries = {}
negs_to_use = None


with gzip.open(hard_negatives_filepath, 'rt') as fIn:
    for line in tqdm.tqdm(fIn):
        if max_passages > 0 and len(train_queries) >= max_passages:
            break
        data = json.loads(line)

        #Get the positive passage ids
        pos_pids = data['pos']

        if len(pos_pids) == 0:  #Skip entries without positives passages
            continue

        pos_min_ce_score = min([ce_scores[data['qid']][pid] for pid in data['pos']])
        ce_score_threshold = pos_min_ce_score - ce_score_margin

        neg_pids = []
        
        #Get the hard negatives
        
        if negs_to_use is None:
            negs_to_use = ['splade']
            
        for system_name in negs_to_use:
            if system_name not in data['neg']:
                continue

            system_negs = data['neg'][system_name]
            negs_added = 0
            for pid in system_negs:
                pid = int(pid)
                if pid not in neg_pids:
                    neg_pids.append(pid)
                    negs_added += 1
                    if negs_added >= num_negs_per_system:
                        break

        if ((len(pos_pids) > 0 and len(neg_pids) > 0)):
            train_queries[data['qid']] = {'qid': data['qid'], 'query': queries[data['qid']], 'pos': pos_pids, 'neg': neg_pids}

for qid in train_queries:          
    pos_list = []
    for posid in train_queries[qid]['pos']:
        if posid in train_queries[qid]['neg']:
            pos_list.append([train_queries[qid]['neg'].index(posid) + 1, posid])
        else:
            pos_list.append([len(train_queries[qid]['neg']) + 1, posid])

    target_scores = [[pid[1], ce_scores[qid][pid[1]]] for pid in pos_list] + [[pid, ce_scores[qid][pid]] for pid in train_queries[qid]['neg']]
    target_scores = sorted(target_scores, key = lambda x: -x[1])
    target_ids = [x[0] for x in target_scores]

    train_queries[qid]['pos'] = [x + [target_ids.index(x[1]) + 1] for x in pos_list]
    train_queries[qid]['neg'] = [[x[0] + 1, x[1], target_ids.index(x[1]) + 1] for x in enumerate(train_queries[qid]['neg'])]
    train_queries[qid]['slade_pos'] = [x + [target_ids.index(x[1]) + 1] for x in pos_list]
    train_queries[qid]['splade_neg'] = [[x[0] + 1, x[1], target_ids.index(x[1]) + 1] for x in enumerate(train_queries[qid]['neg'])]
            

In [ ]:
from collections import defaultdict
import pytrec_eval
from statistics import mean
import scipy.stats as stats

from collections import defaultdict
qrel_file = "../msmarco/qrels.dev.tsv"

qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)

VALIDATION_METRIC = 'recip_rank' 

top_run = defaultdict(dict)

with open("../msmarco/yifan_bm25_top1000.psg.dev.trec.trec") as f:
    for line in f:
        qid, _, did, rank, score, _ = line.strip().split("\t")
        top_run[qid][did] = float(score)
        
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
eval_scores = trec_eval.evaluate(top_run)
print(mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))

In [ ]:
top_run['1048585']['7187158']

In [ ]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from collections import defaultdict
from tqdm import tqdm
import os
import pytrec_eval
from collections import OrderedDict, defaultdict
from statistics import mean
import json
import sys
from training_with_sentence_transformers.models import MLMTransformer
from training_with_sentence_transformers.losses import pairwise_dot_score
import nvidia_smi
from sentence_transformers import  models as smodel
import torch.nn as nn
from sentence_transformers import SentenceTransformer

def _split_into_batches(features, bsize):
    batches = []
    for offset in range(0, features["input_ids"].size(0), bsize):
        batches.append({key: features[key][offset:offset+bsize] for key in features.keys()})

    return batches

nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

agg = "max"

model_type_or_dir = "training_with_sentence_transformers/output/distilSplade_lastquerylayer_lambdauni-0.5_lr-0.001_batch_size_128-2022-09-09_12-41-45/200"


topk = open("../msmarco/msmarco-passagetest2019-top1000.tsv") #open("../msmarco/wentai_splade_dev_top1000.tsv")

VALIDATION_METRIC = 'recip_rank'   #'recip_rank' #'ndcg_cut_10' 

qrel_file = "../msmarco/2019qrels-pass.txt" #"../msmarco/qrels.dev.tsv"

qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)


class Splade(torch.nn.Module):

    def __init__(self, model_type_or_dir, agg="max"):
        super().__init__()
        self.transformer = AutoModelForMaskedLM.from_pretrained(model_type_or_dir)
        assert agg in ("sum", "max")
        self.agg = agg
    
    def forward(self, **kwargs):
        out = self.transformer(**kwargs)["logits"] # output (logits) of MLM head, shape (bs, pad_len, voc_size)
        if self.agg == "max":
            values, _ = torch.max(torch.log(1 + torch.relu(out)) * kwargs["attention_mask"].unsqueeze(-1), dim=1)
            return values
            # 0 masking also works with max because all activations are positive
        else:
            return torch.sum(torch.log(1 + torch.relu(out)) * kwargs["attention_mask"].unsqueeze(-1), dim=1)


model0 = Splade(os.path.join(model_type_or_dir, "0_MLMTransformer"), agg=agg)
d = 30522
model1 = smodel.Dense(in_features=d, out_features = d)

checkpoint = torch.load(os.path.join(model_type_or_dir, "1_Dense", "pytorch_model.bin"), map_location='cpu')
model1.load_state_dict(checkpoint)


model0.eval()
model0.to("cuda")

model1.eval()
model1.to("cuda")


In [ ]:
#FROM Sentence-BERT(https://github.com/UKPLab/sentence-transformers/blob/afee883a17ab039120783fd0cffe09ea979233cf/examples/training/ms_marco/train_bi-encoder_margin-mse.py) with minimal changes.
#Original License APACHE2

from torch import nn
from transformers import AutoModel, AutoModelForMaskedLM, AutoTokenizer, AutoConfig
import json
from typing import List, Dict, Optional, Union, Tuple
import os
import torch
from torch import Tensor
import torch.nn.functional as F

class Splade_Pooling(nn.Module):
    def __init__(self, word_embedding_dimension: int):
        super(Splade_Pooling, self).__init__()
        self.word_embedding_dimension = word_embedding_dimension
        self.config_keys = ["word_embedding_dimension"]

    def __repr__(self):
        return "Pooling Splade({})"

    def get_pooling_mode_str(self) -> str:
        return "Splade"

    def forward(self, features: Dict[str, Tensor]):
        token_embeddings = features['token_embeddings']
        attention_mask = features['attention_mask']

        ## Pooling strategy
        output_vectors = []
        sentence_embedding = torch.max(torch.log(1 + torch.relu(token_embeddings)) * attention_mask.unsqueeze(-1), dim=1).values
        features.update({'sentence_embedding': sentence_embedding})
        return features

    def get_sentence_embedding_dimension(self):
        return self.word_embedding_dimension

    def get_config_dict(self):
        return {key: self.__dict__[key] for key in self.config_keys}

    def save(self, output_path):
        with open(os.path.join(output_path, 'config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(), fOut, indent=2)

    @staticmethod
    def load(input_path):
        with open(os.path.join(input_path, 'config.json')) as fIn:
            config = json.load(fIn)

        return Splade_Pooling(**config)

class MLMTransformer(nn.Module):
    """Huggingface AutoModel to generate token embeddings.
    Loads the correct class, e.g. BERT / RoBERTa etc.

    :param model_name_or_path: Huggingface models name (https://huggingface.co/models)
    :param max_seq_length: Truncate any inputs longer than max_seq_length
    :param model_args: Arguments (key, value pairs) passed to the Huggingface Transformers model
    :param cache_dir: Cache dir for Huggingface Transformers to store/load models
    :param tokenizer_args: Arguments (key, value pairs) passed to the Huggingface Tokenizer model
    :param do_lower_case: If true, lowercases the input (independent if the model is cased or not)
    :param tokenizer_name_or_path: Name or path of the tokenizer. When None, then model_name_or_path is used
    """
    def __init__(self, model_name_or_path: str, max_seq_length: Optional[int] = None,
                 model_args: Dict = {}, cache_dir: Optional[str] = None,
                 tokenizer_args: Dict = {}, do_lower_case: bool = False,
                 tokenizer_name_or_path : str = None):
        super(MLMTransformer, self).__init__()
        self.config_keys = ['max_seq_length', 'do_lower_case']
        self.do_lower_case = do_lower_case

        config = AutoConfig.from_pretrained(model_name_or_path, **model_args, cache_dir=cache_dir)
        self.auto_model = torch.nn.DataParallel(AutoModelForMaskedLM.from_pretrained(model_name_or_path, config=config, cache_dir=cache_dir))
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path if tokenizer_name_or_path is not None else model_name_or_path, cache_dir=cache_dir, **tokenizer_args)
        self.pooling = torch.nn.DataParallel(Splade_Pooling(self.get_word_embedding_dimension())) 
        
        # No max_seq_length set. Try to infer from model
        if max_seq_length is None:
            if hasattr(self.auto_model, "config") and hasattr(self.auto_model.config, "max_position_embeddings") and hasattr(self.tokenizer, "model_max_length"):
                max_seq_length = min(self.auto_model.config.max_position_embeddings, self.tokenizer.model_max_length)

        self.max_seq_length = max_seq_length

        if tokenizer_name_or_path is not None:
            self.auto_model.config.tokenizer_class = self.tokenizer.__class__.__name__

    def __repr__(self):
        return "MLMTransformer({}) with Transformer model: {} ".format(self.get_config_dict(), self.auto_model.__class__.__name__)

    def forward(self, features):
        """Returns token_embeddings, cls_token"""
        trans_features = {'input_ids': features['input_ids'], 'attention_mask': features['attention_mask']}
        if 'token_type_ids' in features:
            trans_features['token_type_ids'] = features['token_type_ids']

        output_states = self.auto_model(**trans_features, return_dict=False)
        output_tokens = output_states[0]

        features.update({'token_embeddings': output_tokens, 'attention_mask': features['attention_mask']})

        if self.auto_model.module.config.output_hidden_states:
            all_layer_idx = 2
            if len(output_states) < 3: #Some models only output last_hidden_states and all_hidden_states
                all_layer_idx = 1

            hidden_states = output_states[all_layer_idx]
            features.update({'all_layer_embeddings': hidden_states})

        features = self.pooling(features)

        return features

    def get_word_embedding_dimension(self) -> int:
            return self.auto_model.module.config.vocab_size
        
    def tokenize(self, texts: Union[List[str], List[Dict], List[Tuple[str, str]]]):
        """
        Tokenizes a text and maps tokens to token-ids
        """
        output = {}
        if isinstance(texts[0], str):
            to_tokenize = [texts]
        elif isinstance(texts[0], dict):
            to_tokenize = []
            output['text_keys'] = []
            for lookup in texts:
                text_key, text = next(iter(lookup.items()))
                to_tokenize.append(text)
                output['text_keys'].append(text_key)
            to_tokenize = [to_tokenize]
        else:
            batch1, batch2 = [], []
            for text_tuple in texts:
                batch1.append(text_tuple[0])
                batch2.append(text_tuple[1])
            to_tokenize = [batch1, batch2]

        #strip
        to_tokenize = [[str(s).strip() for s in col] for col in to_tokenize]

        #Lowercase
        if self.do_lower_case:
            to_tokenize = [[s.lower() for s in col] for col in to_tokenize]

        output.update(self.tokenizer(*to_tokenize, padding=True, truncation='longest_first', return_tensors="pt", max_length=self.max_seq_length))
        return output

    def get_config_dict(self):
        return {key: self.__dict__[key] for key in self.config_keys}

    def save(self, output_path: str):
        self.auto_model.module.save_pretrained(output_path)
        self.tokenizer.save_pretrained(output_path)

        with open(os.path.join(output_path, 'sentence_bert_config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(), fOut, indent=2)

    @staticmethod
    def load(input_path: str):
        #Old classes used other config names than 'sentence_bert_config.json'
        for config_name in ['sentence_bert_config.json', 'sentence_roberta_config.json', 'sentence_distilbert_config.json', 'sentence_camembert_config.json', 'sentence_albert_config.json', 'sentence_xlm-roberta_config.json', 'sentence_xlnet_config.json']:
            sbert_config_path = os.path.join(input_path, config_name)
            if os.path.exists(sbert_config_path):
                break

        with open(sbert_config_path) as fIn:
            config = json.load(fIn)
        return MLMTransformer(model_name_or_path=input_path, **config)

class Discriminator(nn.Module):
    def __init__(self, in_features=30522, out_features=256, num_layers=2, dropout=0.1):
        super(Discriminator, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_layers = num_layers
        hidden_layers = []
        for i in range(num_layers):
            if i == 0:
                input_dim = in_features
            else:
                input_dim = out_features
            hidden_layers.append(nn.Sequential(nn.Linear(input_dim, out_features), nn.GELU(), nn.Dropout(dropout)))

        hidden_layers.append(nn.Linear(out_features, 2))
        self.hidden_layers = nn.ModuleList(hidden_layers)

    def forward(self, features: Dict[str, Tensor]):
        x = features['sentence_embedding']
        for i in range(self.num_layers - 1):
            x = self.hidden_layers[i](x)

        features.update({'sentence_embedding': x})
        return features
    
    def get_sentence_embedding_dimension(self) -> int:
        return self.out_features

    def get_config_dict(self):
        return {'in_features': self.in_features, 'out_features': self.out_features, 'num_layers': self.num_layers}

    def save(self, output_path):
        with open(os.path.join(output_path, 'config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(), fOut)

        torch.save(self.state_dict(), os.path.join(output_path, 'pytorch_model.bin'))

    def __repr__(self):
        return "Discriminator({})".format(self.get_config_dict())

    @staticmethod
    def load(input_path):
        with open(os.path.join(input_path, 'config.json')) as fIn:
            config = json.load(fIn)

        model = Discriminator(**config)
        model.load_state_dict(torch.load(os.path.join(input_path, 'pytorch_model.bin'), map_location=torch.device('cpu')))
        return model

In [ ]:
word_embedding_model = MLMTransformer("training_with_sentence_transformers/output/splade_adv_base-2022-09-14_11-32-24/iter_0/1000/0_MLMTransformer", max_seq_length=256)
dim=word_embedding_model.get_word_embedding_dimension()
discrim = Discriminator(in_features=dim, out_features = 512, num_layers = 2)
discrim = discrim.load("training_with_sentence_transformers/output/splade_adv_base-2022-09-14_11-32-24/iter_0/1000/1_Discriminator")

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, util, evaluation, InputExample
from sentence_transformers import  models as smodel

model = SentenceTransformer(modules=[word_embedding_model, discrim])

In [ ]:
for p in model[0].named_parameters():
    print(p)

In [ ]:
for p in model[0].named_parameters():
    print(p)

In [ ]:
for p in model[1].named_parameters():
    print(p)

In [ ]:
for p in model[1].named_parameters():
    print(p)

In [ ]:
import logging

In [ ]:
def run():
    logging.info("start_logging")

In [ ]:
run()

In [ ]:
a = {"a":1}

In [4]:
import torch

torch.cat([torch.ones(5,dtype=torch.long), torch.ones(5 * 2,dtype=torch.long)])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [6]:
 with open("../msmarco/chatgpt.2019.original") as f, open("../msmarco/queries.2019.tsv") as f1, open("../msmarco/chatgpt.2019.original.expand", "w") as fo:
    for line in f:
        qid, expand = line.split()[0], " ".join(line.strip().split()[1:])
        qid, qtext = f1.readline().strip().split("\t")
        fo.write(f"{qid}\t{qtext}. {expand}\n")

['131843',
 'A',
 'SIGMET',
 'is',
 'a',
 'notice',
 'of',
 'significant',
 'hazardous',
 'weather',
 'conditions',
 'in',
 'a',
 'large',
 'area',
 'that',
 'is',
 'provided',
 'to',
 'the',
 'pilot',
 'of',
 'an',
 'aircraft',
 'before',
 'takeoff.',
 'These',
 'conditions',
 'may',
 'include',
 'extreme',
 'turbulence,',
 'icing,',
 'or',
 'poor',
 'visibility.',
 'SIGMETs',
 'are',
 'issued',
 'to',
 'warn',
 'pilots',
 'of',
 'potentially',
 'hazardous',
 'weather',
 'that',
 'may',
 'impact',
 'their',
 'flight.']

In [7]:
import gzip, json,os

In [12]:
out_dir = "training_with_sentence_transformers/output/colbert_cont_128_num20_kldiv_ib5-lr1e-05-batch_size_32x1-2023-04-17/index"
with gzip.open(os.path.join(out_dir, f"file_0_unorm.jsonl.gz")) as f:
    for line in f:
        break


In [10]:
from transformers import AutoTokenizer
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained('training_with_sentence_transformers/output/colbert_cont_128_num20_kldiv_ib5-lr1e-05-batch_size_32x1-2023-04-17/236000/0_ColBERTTransformer')

doc_tokens_mapping = dict()
with open("../msmarco/collection.tsv") as f:
    i = 0
    for line in tqdm(f):
        did, doc = line.strip().split("\t")   
        d_features = tokenizer("[unused1] " + doc, return_tensors="pt", truncation=True)['input_ids'].tolist()[0]   
        doc_tokens_mapping[i] = d_features
        i += 1
        if i > 100:
            break

100it [00:00, 3223.73it/s]


In [8]:
non_contextual_emb_dict = dict()

with gzip.open(os.path.join(out_dir, f"file_static_embedding.jsonl.gz")) as f:
    for line in f:
        info = json.loads(line)
        non_contextual_emb_dict[info['id']] = info['content'][0]


In [13]:

import torch
orig_emb = []
from tqdm import tqdm
import os,json,gzip
with gzip.open(os.path.join(out_dir, "file_0_unorm.jsonl.gz")) as f:
    for line in tqdm(f):
        info = json.loads(line)
        tokens = doc_tokens_mapping[info['id']]
        non_context = [non_contextual_emb_dict[t] for t in tokens]
        doc_emb = info['content'][0]
        orig_emb.append(torch.tensor([x + y for x,y in zip(doc_emb, non_context)]))
        break

0it [00:00, ?it/s]


In [17]:
import torch
orig_emb = torch.cat(orig_emb, 0)

In [18]:
torch.save(orig_emb, os.path.join(out_dir,"orig_embedding.pt"))

In [17]:
orig_emb[0]

tensor([[-0.5840,  0.5057,  0.4397,  ...,  0.2244,  0.1074, -0.3608],
        [ 1.2291, -1.8514, -1.7388,  ..., -1.2743,  1.3311,  1.4852],
        [ 0.0168,  0.7232,  0.0705,  ..., -0.1868,  0.5319, -0.3616],
        ...,
        [-0.1895,  0.4522, -0.5263,  ..., -1.3590,  1.4736,  1.5435],
        [-0.1313,  0.0546, -0.2861,  ..., -1.4468,  1.5195,  1.4211],
        [-0.6109,  0.3910,  0.4135,  ..., -0.3466, -0.3173, -0.0903]])

In [25]:
[torch.dot(torch.tensor(doc_emb[i]), torch.tensor(non_context[i])) for i in range(len(doc_emb))]

[tensor(-2.4647),
 tensor(289.7617),
 tensor(1.3801),
 tensor(19.6986),
 tensor(3.2145),
 tensor(22.3680),
 tensor(14.9667),
 tensor(22.8963),
 tensor(23.4582),
 tensor(5.6331),
 tensor(30.3596),
 tensor(28.1236),
 tensor(4.7031),
 tensor(1.2721),
 tensor(29.2605),
 tensor(3.0659),
 tensor(1.6658),
 tensor(25.5213),
 tensor(28.9515),
 tensor(9.0393),
 tensor(21.3529),
 tensor(29.7103),
 tensor(6.4048),
 tensor(17.1815),
 tensor(1.3080),
 tensor(14.0413),
 tensor(20.3873),
 tensor(18.7493),
 tensor(13.9346),
 tensor(2.1581),
 tensor(24.3200),
 tensor(21.8502),
 tensor(4.2137),
 tensor(1.9294),
 tensor(24.8555),
 tensor(21.9800),
 tensor(5.6090),
 tensor(20.5186),
 tensor(4.6384),
 tensor(-3.0557),
 tensor(5.1682),
 tensor(26.0797),
 tensor(24.9808),
 tensor(21.3130),
 tensor(22.8780),
 tensor(10.5159),
 tensor(4.1978),
 tensor(9.5954),
 tensor(3.8435),
 tensor(16.1719),
 tensor(13.4765),
 tensor(17.2271),
 tensor(6.1329),
 tensor(18.6621),
 tensor(20.9220),
 tensor(-12.9603)]

In [26]:
doc_norm = torch.nn.functional.normalize(torch.tensor(doc_emb))
doc_static_norm = torch.nn.functional.normalize(torch.tensor(non_context))

In [33]:
torch.sum(doc_norm * doc_static_norm, 1)

tensor([-0.0806,  0.9024,  0.0640,  0.6459,  0.1118,  0.6763,  0.5292,  0.6072,
         0.6307,  0.2159,  0.7617,  0.7242,  0.1641,  0.0558,  0.7142,  0.1114,
         0.0724,  0.7620,  0.7525,  0.4128,  0.6507,  0.7184,  0.2323,  0.1980,
         0.0590,  0.4218,  0.5398,  0.5050,  0.4176,  0.0965,  0.6284,  0.5888,
         0.1570,  0.0823,  0.7454,  0.5300,  0.1222,  0.5827,  0.2076, -0.1084,
         0.2107,  0.7455,  0.6411,  0.5504,  0.3326,  0.3094,  0.1623,  0.2729,
         0.1429,  0.4319,  0.4029,  0.4494,  0.2142,  0.1805,  0.3191, -0.1401])

In [40]:
doc_emb

[[-0.5839661359786987,
  0.5056681036949158,
  0.4396858811378479,
  1.148500919342041,
  -0.0636197179555893,
  0.0005307774990797043,
  -0.5026658773422241,
  0.30251285433769226,
  -0.09085653722286224,
  -1.156367301940918,
  -0.25769364833831787,
  0.03184965252876282,
  0.7407220602035522,
  0.0881834551692009,
  0.4040738046169281,
  0.09802064299583435,
  0.14264363050460815,
  -0.15015678107738495,
  -0.48674720525741577,
  -0.3306870460510254,
  -0.4199691414833069,
  0.822181761264801,
  -0.06016537547111511,
  1.1002310514450073,
  0.12689360976219177,
  0.0254807285964489,
  -0.02996034361422062,
  0.928370475769043,
  0.19800950586795807,
  1.0847080945968628,
  -0.4642026722431183,
  0.13920903205871582,
  -0.6429795026779175,
  0.36477577686309814,
  0.6693696975708008,
  0.4524526596069336,
  -1.0270462036132812,
  0.17227675020694733,
  -0.5935448408126831,
  -0.3287559747695923,
  0.23891650140285492,
  -0.2310228943824768,
  -0.3191659450531006,
  -0.316253095865249

In [2]:
import torch, os
out_dir = "training_with_sentence_transformers/output/colbert_cont_128_num20_kldiv_ib5-lr1e-05-batch_size_32x1-2023-04-17/index"

orig_emb = torch.load(os.path.join(out_dir, f"orig_embedding.pt"))

In [19]:
tokenizer.mask_token_id

103

In [21]:
tokenizer(["[unused0] I love my mom", "[unused0] I love my mom and dad"], return_tensors="pt", padding = True)

{'input_ids': tensor([[ 101,    1, 1045, 2293, 2026, 3566,  102,    0,    0],
        [ 101,    1, 1045, 2293, 2026, 3566, 1998, 3611,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
tokenizer.

0

In [4]:
import os
from collections import defaultdict
#### Read the corpus file containing all the passages. Store them in the corpus dict
corpus = {}  # dict in the format: passage_id -> passage. Stores all existing passages
data_folder = "../msmarco/"
collection_filepath = os.path.join(data_folder, 'collection.tsv')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[pid] = passage

### Read the train queries, store in queries dict
queries = {}  # dict in the format: query_id -> query. Stores all training queries
queries_filepath = os.path.join(data_folder, 'queries.2019.tsv') #'queries.train.tsv'  'queries.train.qspladev2.top10.tsv'  'queries.train.spladedoc3_10.tsv'

with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query


In [2]:
qmapping = defaultdict(list)
all_did = set()
with open("../msmarco/effi_V_19.trec") as f:
    for line in f:
        qid, _, did, rank, splade_score , _ = line.split("\t")
        qmapping[qid].append(did)
        all_did.add(did)

In [7]:
with open("../msmarco/effi.dev.top100.trec.tsv", "w") as f:
    for q in qmapping:
        for d in qmapping[q][:100]:
            f.write("\t".join([q,d,queries[q],corpus[d]]) + "\n")

In [5]:
with open("../msmarco/effi_V_19.trec.tsv", "w") as f:
    for q in qmapping:
        for d in qmapping[q]:
            f.write("\t".join([q,d,queries[q],corpus[d]]) + "\n")